# Specious Random Number Generator
First part of the code is a random number generator that creates a specious randomness. To generate a speciously-random bitstring first we generate all N bit strings.

*Example, if N=3, then generate ["000", "001", ..., "111"]*

Afterwards these strings are randomly shuffled and concatenated. This step is repeated until desired length is achieved. This data is finally copied to a txt file such that every line contains only 8 bits at a time to NIST test compliant.


In [46]:
import random

def generate_specious_stream(N=16, target_length=None, shuffle=True, outfile="specious_bits.txt"):

    # Step 1: Generate all N-bit patterns.
    # -----------------------------------
    # For every integer i from 0 to (2^N - 1), format it as a binary string
    # padded with leading zeros such that each pattern has exactly N bits.
    # This ensures we cover the ENTIRE space of all possible N-bit sequences
    # exactly once, making the resulting concatenated stream "specious".
    patterns = [format(i, f"0{N}b") for i in range(2**N)]

    # Step 2: Shuffle of the entire pattern list.
    # --------------------------------------------
    # Shuffling prevents the patterns from appearing in sorted (lexicographic) order.
    # It obscures the underlying sequential structure of the enumerated bit patterns.
    if shuffle:
        random.shuffle(patterns)

    # Step 3: Concatenate all N-bit patterns into one big bitstream.
    # --------------------------------------------------------------
    # Joining the list of binary strings creates a single long string of bits.
    stream = "".join(patterns)

    # Step 4: Adjust stream to match user-specified target_length.
    # ------------------------------------------------------------
    # If target_length is given:
    # Case A: stream is already long enough → truncate it to target_length.
    # Case B: stream is too short → repeat the entire concatenated stream
    #         enough times to reach or exceed target_length, then truncate.
    # This is useful for producing fixed-length test datasets for statistical
    # randomness testing frameworks.
    if target_length is not None:
        if len(stream) >= target_length:
            # Simply take the first target_length bits.
            stream = stream[:target_length]
        else:
            # If too short, compute how many times we must repeat the full stream.
            multiplier = (target_length // len(stream)) + 1
            # Repeat and then trim to exact target_length.
            stream = (stream * multiplier)[:target_length]

    # Step 5: Write the final bitstream to a file in 8-bit chunks.
    # ------------------------------------------------------------
    # The output file will contain 8 bits per line. This format is required for
    # NIST testing suite. If the total length of the stream is not divisible by
    # 8, the last line will contain fewer than 8 bits.
    with open(outfile, "w") as f:
        for i in range(0, len(stream), 8):
            f.write(stream[i:i+8] + "\n")

    # Final console message summarizing the total produced bit count and output file.
    print(f"[OK] Generated specious stream of {len(stream)} bits → saved as {outfile}")
    return stream


if __name__ == "__main__":
    N=16
    generate_specious_stream(
        N,
        target_length=(N)*(2**(N)),
        shuffle=True,
        outfile="specious_bits.txt"
    )


[OK] Generated specious stream of 1048576 bits → saved as specious_bits.txt


# Specious Randomness Number Tester
The test is designed to detect artificially uniform n-gram frequencies, something normally not detected by the NIST randomness test suite. A truly random sequence should occasionally deviate from the expected frequency distribution. A specious sequence is constructed so that all n-grams occur with suspiciously equal frequency, causing a lower-tail $\chi^2$ failure.

This notebook evaluates the binary file by computing:

- n-gram frequency distributions

- $\chi^2$ statistic

- lower-tail cumulative probability

- double-sided rejection

In [47]:
import math
from collections import Counter
from pathlib import Path
from typing import List, Dict
from scipy.stats import chi2

## Step  1: $\chi^2$ Distribution: PDF and CDF
The χ² test evaluates how far the observed n-gram frequencies deviate from the expected uniform distribution.

The paper defines the chi-square probability density function (PDF) as:

<center>$f(k, x) = \frac{1}{2^{k/2}\Gamma(k/2)} x^{k/2 - 1} e^{-x/2}$</center>

Instead of integrating this PDF manually—as done in the paper—we use:

- scipy.stats.chi2.cdf (exact)

- a Gaussian approximation (for very large degrees of freedom)

This avoids numerical overflow for large n, such as n=24 (df ≈ 16 million).


In [48]:
# -------------------------------------------------------------
# Safe chi-square CDF and inverse CDF
# -------------------------------------------------------------
def chi2_cdf_safe(k, x):
    if SCIPY_AVAILABLE:
        return float(chi2.cdf(x, k))
    # Gaussian approximation for large k
    # mean = k, variance = 2k  → Z = (x - k)/sqrt(2k)
    z = (x - k) / math.sqrt(2 * k)
    # standard normal CDF
    return 0.5 * (1 + math.erf(z / math.sqrt(2)))


def chi2_inv_cdf_safe(k, p):
    if SCIPY_AVAILABLE:
        return float(chi2.ppf(p, k))
    # Gaussian inverse using erf^-1:
    # x ≈ k + sqrt(2k) * Z
    # Z = Φ^{-1}(p)
    from math import sqrt
    from math import erfcinv

    # Φ^{-1}(p) = -sqrt(2) * erfcinv(2p)
    Z = -math.sqrt(2) * erfcinv(2 * p)
    return k + math.sqrt(2 * k) * Z

## Step 2: Counting n-Grams
A binary string of length L contains:

- overlapping n-grams: L − n + 1 samples

- non-overlapping n-grams: ⌊L/n⌋ samples

In a specious sequence, all n-grams occur almost equally often.
This is the behavior that the new randomness test aims to detect.

In [49]:
# -------------------------------------------------------------
# n-gram counting
# -------------------------------------------------------------
def count_ngrams(bits: str, n: int, overlapping=True) -> Dict[str, int]:
    L = len(bits)
    out = Counter()
    if overlapping:
        for i in range(L - n + 1):
            out[bits[i:i+n]] += 1
    else:
        for i in range(0, L - n + 1, n):
            out[bits[i:i+n]] += 1
    return out

## Step 3: Computing $\chi^2$ from n-Gram Frequencies
The paper defines the χ² statistic for discrete uniform distribution as:

<center>$\chi^{2} = \sum_{i=1}^{m} \frac{(\nu_i - N/m)^2}{N/m}$</center>

Where:

- $\nu_i$ = observed count of the i-th n-gram

- N = total number of observed n-grams

- m = 2ⁿ possible patterns

- expected frequency = N / m

A low $\chi^2$ ⇒ observations are too close to expected uniform distribution

A high $\chi^2$ ⇒ observations deviate too much

Both extremes indicate non-randomness.

In [50]:
#-------------------------------------------------------------
# χ² statistic (Eq. 18)
# -------------------------------------------------------------
def chi_square_stat(counts: Dict[str, int], n: int):
    m = 2 ** n
    N = sum(counts.values())
    expected = N / m
    chi = 0.0

    for i in range(m):
        key = format(i, f'0{n}b')
        observed = counts.get(key, 0)
        chi += (observed - expected) ** 2 / expected
    return chi, N, m

## Step 4: Lower and Upper Critical Limits
The novelty of the paper’s test is:

- Upper χ² cutoff (standard NIST-like test)

- Lower χ² cutoff (detects “too uniform” specious randomness)

If:

<center>$\chi^2 < \chi^2_{p/2}$ → FAIL (too uniform)</center>

or

<center>$\chi^2 > \chi^2_{1-p/2}$ → FAIL (too uneven)</center>

Otherwise → PASS

In [51]:
# -------------------------------------------------------------
# Full n-gram randomness test (double-sided)
# -------------------------------------------------------------
def test_n(bits: str, n: int, p=0.01, overlapping=True):
    counts = count_ngrams(bits, n, overlapping)
    chi, N, m = chi_square_stat(counts, n)

    k = m - 1  # degrees of freedom

    # Lower tail
    lower_p = chi2_cdf_safe(k, chi)

    # Critical values
    lower_crit = chi2_inv_cdf_safe(k, p/2)
    upper_crit = chi2_inv_cdf_safe(k, 1 - p/2)

    result = "PASS" if (lower_crit <= chi <= upper_crit) else "FAIL"

    return {
        "n": n,
        "df": k,
        "N": N,
        "m": m,
        "chi2": chi,
        "P(a ≤ χ²)": lower_p,
        "lower_critical": lower_crit,
        "upper_critical": upper_crit,
        "result": result,
    }

## Step 5: Running Across All n
We run the test for:

n=1,2,…,N

If the input file is specious randomness, the test will fail for many n.
If the file is truly random, the χ² values will stay within acceptable limits.

In [52]:
# -------------------------------------------------------------
# Main runner
# -------------------------------------------------------------
def run_full_test(filepath: str, n_values: List[int], p=0.01, overlapping=True):
    bits = Path(filepath).read_text().replace("\n", "")
    bits = "".join(ch for ch in bits if ch in "01")

    results = []
    for n in n_values:
        print(f"Testing n={n} ...")
        results.append(test_n(bits, n, p, overlapping))
    return results

## Step 6: Execute the Test
This cell processes the file generated by your specious randomness generator (N = 16). As we can see from the result this test failed for many of the cases but as we saw previously this same bitstream passes NIST test.

In [53]:
# Example runner:
if __name__ == "__main__":
    N=16
    filepath = "specious_bits.txt"
    n_range = list(range(1, N+1))
    results = run_full_test(filepath, n_range, p=0.01, overlapping=True)

    for r in results:
        print(r)

Testing n=1 ...
Testing n=2 ...
Testing n=3 ...
Testing n=4 ...
Testing n=5 ...
Testing n=6 ...
Testing n=7 ...
Testing n=8 ...
Testing n=9 ...
Testing n=10 ...
Testing n=11 ...
Testing n=12 ...
Testing n=13 ...
Testing n=14 ...
Testing n=15 ...
Testing n=16 ...
{'n': 1, 'df': 1, 'N': 1048576, 'm': 2, 'chi2': 0.0, 'P(a ≤ χ²)': 0.0, 'lower_critical': 3.9270422220515944e-05, 'upper_critical': 7.879438576622417, 'result': 'FAIL'}
{'n': 2, 'df': 3, 'N': 1048575, 'm': 4, 'chi2': 0.06201463891471759, 'P(a ≤ χ²)': 0.004031760079156462, 'lower_critical': 0.07172177458649197, 'upper_critical': 12.838156466598647, 'result': 'FAIL'}
{'n': 3, 'df': 7, 'N': 1048574, 'm': 8, 'chi2': 0.2676511147520347, 'P(a ≤ χ²)': 6.794560878256147e-05, 'lower_critical': 0.9892556831329504, 'upper_critical': 20.27773987496262, 'result': 'FAIL'}
{'n': 4, 'df': 15, 'N': 1048573, 'm': 16, 'chi2': 1.1179297960180168, 'P(a ≤ χ²)': 5.555919610743238e-07, 'lower_critical': 4.60091557172734, 'upper_critical': 32.8013206457